<a href="https://colab.research.google.com/github/stmulugheta/AI-Projects-2021/blob/main/Join_IGRA_ERA_datasets_checkpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from pathlib import Path

import pandas as pd

## Load the raw data from ApprovedDatasets folder

In [ ]:
# get parent directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

#get data directory
ERA_path = os.path.join(parent_dir, "ApprovedDatasets", "ERA5_InitialRaw", "ERA5_InitialRaw_iokolab_EspoirGaglo.csv")
IGRA_path = os.path.join(parent_dir, "ApprovedDatasets", "IGRA_InitialRaw", "IGRA2_InitialRaw_RikDutta.csv")

In [ ]:
ERA_df = pd.read_csv(ERA_path, index_col=0)
IGRA_df = pd.read_csv(IGRA_path)

## Check if the dataframes seem correct

In [ ]:
ERA_df.head()

,"coordinates (lat,lon)",model (name),model elevation (surface),utc_offset (hrs),temperature (degC),wind_speed (m/s),relative_humidity (0-1),surface_solar_radiation (W/m^2),dewpoint_temperature (degC),total_precipitation (mm of water equivalent)
2008-01-01 00:00:00,"(4.05, 9.7)",era5,35.73,1.0,25.26,0.92,0.94,0.0,24.27,0.11
2008-01-01 01:00:00,"(4.05, 9.7)",era5,35.73,1.0,25.03,0.67,0.95,0.0,24.16,0.07
2008-01-01 02:00:00,"(4.05, 9.7)",era5,35.73,1.0,24.62,0.13,0.95,0.0,23.76,0.39
2008-01-01 03:00:00,"(4.05, 9.7)",era5,35.73,1.0,24.59,0.52,0.96,0.0,23.87,0.05
2008-01-01 04:00:00,"(4.05, 9.7)",era5,35.73,1.0,24.61,0.60,0.96,0.0,23.89,0.20


In [ ]:
IGRA_df.head()

,year,month,day,hour,l1,l2,etime,pressure,pressureFlag,geopotentialHeight,zflag,temperature,temperatureProcessingFlag,relativeHumidity,dewpointDepression,windDirection,windSpeed
0,2008,1,1,0,2,1,NaN,101000.0,B,NaN,,26.0,B,94.796229,0.9,0.0,0.0
1,2008,1,1,0,1,0,NaN,100000.0,,89.0,B,26.4,B,95.943519,0.7,145.0,26.0
2,2008,1,1,0,2,0,NaN,99600.0,,NaN,,26.8,B,94.826278,0.9,NaN,NaN
3,2008,1,1,0,2,0,NaN,95100.0,,NaN,,NaN,,NaN,NaN,215.0,51.0
4,2008,1,1,0,2,0,NaN,93900.0,,NaN,,23.2,B,100.000000,0.0,NaN,NaN


## Let's create a column with date and time for IGRA

In [ ]:
IGRA_df["hour"].value_counts()

12    154551
0     137544
18      5673
6       3503
15       323
23       263
11       139
Name: hour, dtype: int64

In [ ]:
#We should probably consider the launch at 12
IGRA_df = IGRA_df[IGRA_df["hour"] == 12]

In [ ]:
#creation of datetime column
IGRA_df["date"] = pd.to_datetime(IGRA_df[["year", "month", "day", "hour"]])
IGRA_df = IGRA_df.drop(columns = ["year", "month", "day", "hour"])

## We need to join these two dataframes based on the date

Assumptions for joining the two datasets
- We just need to consider the rainfall for the 12 hours after each launch (https://omdena-kanda.slack.com/archives/C02DFRDENCA/p1632749219074400)
- Any amount of rain during the 12 hours count as a rain event (https://omdena-kanda.slack.com/archives/C02DFRDENCA/p1632854037087300?thread_ts=1632684194.062400&cid=C02DFRDENCA)
- We only need to consider year 2008 to 2020 (https://omdena-kanda.slack.com/archives/C02DFRDENCA/p1632437804014500)
- We can drop rows with pressure values outside 1020 - 200 hpa range (https://omdena-kanda.slack.com/archives/C02F2D1NWRM/p1632572875057700)

In [ ]:
#remove rows in ERA between 00:00 and 12:00
ERA_df["date"] = ERA_df.index.to_list()
ERA_df["date"] = pd.to_datetime(ERA_df["date"])
ERA_df = ERA_df[ERA_df["date"].dt.hour >= 12]

In [ ]:
#convert the datetime to date
ERA_df['date'] = pd.to_datetime(ERA_df['date']).dt.date
#group the days to gether and add the rain values
ERA_df = pd.DataFrame(ERA_df.groupby(['date']).agg({'total_precipitation (mm of water equivalent)':'sum'}))


In [ ]:
no_precipitation = len(ERA_df[ERA_df["total_precipitation (mm of water equivalent)"] == 0])/len(ERA_df)
very_low_precipitation = len(ERA_df[ERA_df["total_precipitation (mm of water equivalent)"] <= 0.5])/len(ERA_df)
low_precipitation = len(ERA_df[ERA_df["total_precipitation (mm of water equivalent)"] <= 1])/len(ERA_df)

print("Ratio of no rain day", no_precipitation)
print("Ratio of very low rain day", very_low_precipitation)
print("Ratio of low rain day", low_precipitation)

Ratio of no rain day 0.0035804549283909013
Ratio of very low rain day 0.06023588879528222
Ratio of low rain day 0.11310025273799494


If we consider that a day without rain is equal to 0 mm of precipitation during the 12 hours then we will only have 0.35% of non rainy day in the data set which seems too low.  

According to https://www.climatestotravel.com/climate/cameroon#douala there is 215 day of rain in a year in Douala Cameroon.  
We probably need to increas the threshold to decide rain/no-rain days


In [ ]:
#converting pressure to hpa
IGRA_df["pressure"] = IGRA_df["pressure"]/100
#removing rows with pressure outside of range 200 - 1020 hpa
IGRA_df = IGRA_df[IGRA_df["pressure"] >= 200]
IGRA_df = IGRA_df[IGRA_df["pressure"] <= 1020]

In [ ]:
print("ERA timespan :", ERA_df.index.min(), "to", ERA_df.index.max())
print("IGRA timespan :", IGRA_df["date"].min(), "to", IGRA_df["date"].max())

ERA timespan : 2008-01-01 00:00:00 to 2020-12-31 00:00:00
IGRA timespan : 2008-01-01 12:00:00 to 2020-12-14 12:00:00
